In [35]:
from lxml import html
import requests
from datetime import datetime
from pprint import pprint

In [40]:
url = 'https://lenta.ru'
header = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.2 Safari/605.1.15'}
response = requests.get(url)


def get_news_lenta_ru():
    
    news = []
    
    keys = ('title', 'date', 'link')
    date_format = '%Y-%m-%dT%H:%M:%S%z'
    link_lenta = 'https://lenta.ru/'
    
    request = requests.get(link_lenta)

    dom = html.fromstring(request.text)
    dom.make_links_absolute(link_lenta)
    
    news_links = dom.xpath('''(//section[@class="row b-top7-for-main js-top-seven"]//div[@class="first-item"]/h2 | 
                                //section[@class="row b-top7-for-main js-top-seven"]//div[@class="item"])
                                /a/@href''')
    
    news_text = dom.xpath('''(//section[@class="row b-top7-for-main js-top-seven"]//div[@class="first-item"]/h2 | 
                                //section[@class="row b-top7-for-main js-top-seven"]//div[@class="item"])
                                /a/text()''')
    
    for i in range(len(news_text)):
        news_text[i] = news_text[i].replace(u'\xa0', u' ')
    
    news_date = []
    
    for item in news_links:
        request = requests.get(item)
        dom = html.fromstring(request.text)
        date = dom.xpath('//time[@itemprop="datePublished"]/@datetime')
        news_date.extend(date)

    for i in range(len(news_date)):
        news_date[i] = datetime.strptime(news_date[i], date_format)
        
    for item in list(zip(news_text, news_date, news_links)):
        news_dict = {}
        for key, value in zip(keys, item):
            news_dict[key] = value
        
        news_dict['source'] = 'lenta.ru'
        news.append(news_dict)
    
    return news

In [41]:
get_news_lenta_ru()

[]